In [127]:
import numpy as np
import pandas as pd

df = pd.read_csv('creditcard.csv')
print(df)

            Time        V1        V2        V3        V4        V5        V6  \
0            0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388   
1            0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361   
2            1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499   
3            1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203   
4            2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921   
...          ...       ...       ...       ...       ...       ...       ...   
284802  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
284803  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
284804  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
284805  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
284806  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [128]:
#checking for null_values
null_values = df.isnull().sum()
#count null values for eachn column
print(null_values)

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [129]:
#the dataset has been sorted  in ascending order by the Class or "label" to be able to get only lables of 0 in the
#train set(this is to help it learn what a "normal" or a 0 should look like) and labels of 1 and 0 in the cross-validation set
#to help test the accuracy of what was trained in the train set by seeing how many anommalies it accurately flags down

In [130]:
#Define training Set
X = df.iloc[:, 1:].values  # All columns except the last one
y = df.iloc[:, -1].values   # Only the last column


# Displaying the shapes of X and y to verify

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (284807, 30)
Shape of y: (284807,)


In [139]:
#Splitting into train set and cross validation set and test set to evaluate the model for high bias or high variance

x_train = X[:284000]
y_train = y[:284000]

x_cv = X[284000:]
y_cv = y[284000:]




print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")



the shape of the training set (input) is: (284000, 30)
the shape of the training set (target) is: (284000,)

the shape of the cross validation set (input) is: (807, 30)
the shape of the cross validation set (target) is: (807,)



In [140]:
def estimate_gaussian(X): 
    """
    Calculates mean and variance of all features 
    in the dataset
    
    Args:
        X (ndarray): (m, n) Data matrix
    
    Returns:
        mu (ndarray): (n,) Mean of all features
        var (ndarray): (n,) Variance of all features
    """

    m, n = X.shape
    mu = np.zeros(n)
    var = np.zeros(n)
    ### START CODE HERE ### 
    for j in range(n):
        #for each feature in all the training examples
        x_feat = X[:,j]
        mu[j] = np.mean(x_feat)
        var[j] =  np.var(x_feat)
    
    
    ### END CODE HERE ### 
        
    return mu, var

In [141]:
# Estimate mean and variance of each feature
mu, var = estimate_gaussian(x_train)              

print("Mean of each feature:", mu)
print("Variance of each feature:", var)

print("Shape of X:", mu.shape)
print("Shape of y:", var.shape)

Mean of each feature: [ 8.46928090e-03 -6.83509843e-03  1.30170501e-02 -7.71014101e-03
  5.17514263e-03  2.50981703e-03  9.48763401e-03 -1.20853186e-03
  4.49318494e-03  9.81603460e-03 -6.13000775e-03  1.05666158e-02
  1.71822994e-04  1.21901600e-02  4.16688647e-04  7.11683274e-03
  1.15556951e-02  3.79485786e-03 -1.11179200e-03 -6.54405721e-04
 -1.27304247e-03 -1.19829121e-04  3.27196085e-05  1.98977978e-04
  5.10346719e-05 -7.91895869e-05 -3.15921256e-04 -1.57065672e-04
  8.83115518e+01  0.00000000e+00]
Variance of each feature: [3.72315286e+00 2.67669761e+00 2.12878675e+00 1.95756230e+00
 1.84171599e+00 1.76853274e+00 1.38981819e+00 1.34857883e+00
 1.18674859e+00 1.08977911e+00 1.00622266e+00 8.95160793e-01
 9.90395720e-01 8.04515554e-01 8.37511627e-01 7.13655025e-01
 5.61648055e-01 6.80606779e-01 6.58849936e-01 5.92279225e-01
 5.14031380e-01 5.23606165e-01 3.86626256e-01 3.66974025e-01
 2.71041838e-01 2.32553563e-01 1.59880835e-01 1.08694158e-01
 6.25976244e+04 0.00000000e+00]
Shap

In [142]:
from scipy.stats import multivariate_normal
#returns the probability of each trainihg example
def multivariate_gaussian(X, mu, var):
    pdf_values = multivariate_normal.pdf(X, mean=mu, cov=var, allow_singular=True)
    return pdf_values
 

p = multivariate_gaussian(x_cv, mu, var)


In [143]:
#select_threshold
#finding a threshold that produces the highest F1 score and balances:
#precision i.e(true positives (predicition and y = 1) in the total amount of flagged positives(i.e predicition = 1))
#recall i.e(true positives(prediction and y =1) in total amount of actual positves (i.e y_val = 1))

def select_threshold(y_val, p_val): 
    """
    Finds the best threshold to use for selecting outliers 
    based on the results from a validation set (p_val) 
    and the ground truth (y_val)
    
    Args:
        y_val (ndarray): Ground truth on validation set
        p_val (ndarray): Results on validation set
        
    Returns:
        epsilon (float): Threshold chosen 
        F1 (float):      F1 score by choosing epsilon as threshold
    """ 

    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    
    step_size = (max(p_val) - min(p_val)) / 1000
    
    for epsilon in np.arange(min(p_val), max(p_val), step_size):
        #iterating through different values of epsilon that range from the minimum y_pred to the
        #maximum y_pred
    
        ### START CODE HERE ### 
        #comparing this particular value of epsilon to the list of our predicted values returning 1 if less than
        #epsilon and 0 if greater than epsilon
        #returns an array of trues and falses i.e 1's and 0's
        predictions = (p_val < epsilon)
        
        #using the predictions to find the number of true positives, false positives and false negatives as shown

        tp = np.sum((predictions == 1) & (y_val == 1))
        fp = np.sum((predictions == 1) & (y_val == 0))
      
        fn = np.sum((predictions == 0) & (y_val == 1))

    
        if tp != 0 and (tp + fp) != 0 and  (tp + fn) != 0:
            precision = tp/(tp + fp)
            recall = tp/(tp + fn)
            F1 = (2*precision*recall)/(precision + recall)
            if F1 > best_F1:
                best_F1 = F1
                best_epsilon = epsilon
      
        
        
        ### END CODE HERE ### 
#       
        
    return best_epsilon, best_F1

In [144]:
epsilon,F1 = select_threshold(y_cv, p)
print(epsilon)
print(F1)

1.1689167343541925e-17
0.8448430493273543


In [146]:
pred =sum(p < epsilon)
actual = sum(y_cv)
print('# Anomalies found: %d'%pred )
print('# Actual Anomalies: %d'%actual)



# Anomalies found: 623
# Actual Anomalies: 492


In [ ]:
#we see that that epsilon is the best threshold value with an error rate of 22% that is
#only 2 in 10 of predicted values were not anomalies